In [1]:
# imports
import pandas as pd
import numpy as np
import glob, os
import matplotlib.pyplot as plt

GPC_SOLVENT = "THF" #"DMF" or "THF"

In [2]:
# fetch excel file names
path = "IJ88"
excel_files = []
for file in os.listdir(path):
    if file.endswith(".xlsx"):
        excel_files.append(os.path.join(path, file))
print(excel_files)

['IJ88/CR.xlsx', 'IJ88/PCL.xlsx', 'IJ88/CA.xlsx', 'IJ88/AN.xlsx', 'IJ88/BC.xlsx', 'IJ88/ProK.xlsx', 'IJ88/PF.xlsx']


In [36]:
# read in excel files into dataframes

if GPC_SOLVENT == "DMF":
    HEADER = 23
elif GPC_SOLVENT == "THF":
    HEADER = 20
else:
    raise Exception("Not a valid GPC Solvent")

dfs = []
for excel_file in excel_files:
    print(excel_file)
    xls = pd.ExcelFile(excel_file)
    dfs.append(pd.read_excel(xls, 'Raw Data', header=HEADER)[HEADER:].reset_index().drop('index', axis=1))

IJ88/CR.xlsx
IJ88/PCL.xlsx
IJ88/CA.xlsx
IJ88/AN.xlsx
IJ88/BC.xlsx
IJ88/ProK.xlsx
IJ88/PF.xlsx


In [27]:
# truncate start data
"""
if GPC_SOLVENT == "DMF":
    RT_COL = 'Response Trace 4'
elif GPC_SOLVENT == "THF":
    RT_COL = 'Response Trace 1'
else:
    raise Exception("Not a valid GPC Solvent")


truncate_index = []
for df in dfs:
    for i in range(len(df)):
        if (df.iloc[i][RT_COL] > -100):
            truncate_index.append(i)
            break
for i in range(len(dfs)):
    dfs[i] = dfs[i][truncate_index[i]:].dropna().reset_index().drop('index', axis=1)
"""

In [37]:
if GPC_SOLVENT == "DMF":
    RT_COL = 'Response Trace 4'
elif GPC_SOLVENT == "THF":
    RT_COL = 'Response Trace 1'
else:
    raise Exception("Not a valid GPC Solvent")

dfs_new = []
for j in range(len(dfs)):
    rt_corrected = []
    for i in range(len(dfs[j])):
        x = dfs[j]['RT (mins)'][i]
        y_curr = dfs[j][RT_COL][i]
        rt_corrected.append( y_curr - (dfs[j].loc[0][RT_COL] + dfs[j].loc[len(dfs[j]) - 1][RT_COL])/(35)*x )
    dfs_new.append(pd.concat([dfs[j], pd.DataFrame({'RT Corrected': rt_corrected})], axis=1, sort=False))

In [38]:
# plot data on separate plots and save as png 
for i in range(len(dfs)):
    plt.figure()
    #plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 1'], label='RT 1')
    #plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 2'], label='RT 2')
    #plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 3'], label='RT 3')
    #plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 4'], 'k-', label='RT 4')
    plt.plot(dfs_new[i]['RT (mins)'], dfs_new[i]['RT Corrected'], 'k-', lw=0.5, label='RT Corrected')
    plt.xlabel("time (mins)")
    plt.ylabel("nRIU")
    #plt.legend()
    #plt.grid()
    plt.savefig(excel_files[i].replace(".xlsx", ""), dpi=300, bbox_inches='tight')
plt.close('all')

In [39]:
# plot all data on 1 overlay plot and save as png 
plt.figure()
plt.xlabel("time (mins)")
plt.ylabel("nRIU")
#plt.grid(b=None)
for i in range(len(dfs)):
    plt.plot(dfs_new[i]['RT (mins)'], dfs_new[i]['RT Corrected'], '-', lw=0.5, label=excel_files[i].replace(".xlsx", ""))
plt.legend()
plt.savefig(path + '/overlay', dpi=300, bbox_inches='tight')
plt.close('all')

In [48]:
# zoomed in region of overlay plot saved as a png 
L_cut = 1200
R_cut = 2100
plt.figure()
plt.xlabel("time (mins)")
plt.ylabel("nRIU")
#plt.grid(b=None)
for i in range(len(dfs)):
    plt.plot(dfs_new[i]['RT (mins)'][L_cut:R_cut], dfs_new[i]['RT Corrected'][L_cut:R_cut], '-', lw=0.5, label=excel_files[i].replace(".xlsx", ""))
plt.legend()
plt.savefig(path + '/zoomed_overlay', dpi=300, bbox_inches='tight')
plt.close('all')

In [44]:
# zoomed in region of overlay plot saved as a png - MANUAL AXIS COLOURS

"""
0 IJ48/IJ48_P5_VD.xlsx
1 IJ48/IJ48_20-50_PVD.xlsx
2 IJ48/IJ48_P5_PVD.xlsx
3 IJ48/IJ48_20-50_SUPER_25.xlsx
4 IJ48/IJ48_20-50_VD.xlsx
5 IJ48/IJ48_20-50_V.xlsx
6 IJ48/IJ48_P5_V.xlsx
"""

L_cut = 2250
R_cut = 3400
plt.figure()
plt.xlabel("time (mins)")
plt.ylabel("nRIU")
#plt.grid(b=None)
plt.plot(dfs_new[2]['RT (mins)'][L_cut:R_cut], dfs_new[2]['RT Corrected'][L_cut:R_cut], 'b-', lw=0.5, label="P5 PVD")
plt.plot(dfs_new[0]['RT (mins)'][L_cut:R_cut], dfs_new[0]['RT Corrected'][L_cut:R_cut], 'b--', lw=0.5, label="P5 VD")
plt.plot(dfs_new[6]['RT (mins)'][L_cut:R_cut], dfs_new[6]['RT Corrected'][L_cut:R_cut], 'b-.', lw=0.5, label="P5 V")
plt.plot(dfs_new[1]['RT (mins)'][L_cut:R_cut], dfs_new[1]['RT Corrected'][L_cut:R_cut], 'g-', lw=0.5, label="20-50 PVD")
plt.plot(dfs_new[4]['RT (mins)'][L_cut:R_cut], dfs_new[4]['RT Corrected'][L_cut:R_cut], 'g--', lw=0.5, label="20-50 VD")
plt.plot(dfs_new[5]['RT (mins)'][L_cut:R_cut], dfs_new[5]['RT Corrected'][L_cut:R_cut], 'g-.', lw=0.5, label="20-50 V")
plt.plot(dfs_new[3]['RT (mins)'][L_cut:R_cut], dfs_new[3]['RT Corrected'][L_cut:R_cut], 'r', lw=0.5, label="20-50 Super")


plt.legend()
plt.savefig(path + '/zoomed_overlay', dpi=300, bbox_inches='tight')
plt.close('all')

In [ ]:
 # integrate zoomed region (i.e. peaks) to get area ratios
for i in range(len(dfs)):
    print(excel_files[i].replace(".xlsx", "") + ' Integrated Peak Area:', np.sum(dfs_new[i]['RT Corrected'][L_cut:R_cut]))

In [ ]:
#IJ3 Specific: take ratio of pure RHP peak : Leftover RHP peak areas
print('% RHP Lost =', (1 - 230695.35791081557/ ((591054.9354821619+540565.0381226097+632872.6023414368)/3))*100 )
  